In [ ]:
from google.colab import drive
from google.colab import files
import requests
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("drive/My Drive/Diploma/second_try/Code/add_category/views_events_1_year_edits_1_day.csv")
# pages.date = pages.date.astype(str)
print(df.shape)
df.head(2)

(108429, 15)


,Unnamed: 0,page_title,date,around,before,boost,same_day_views,prev_day_views,week_before_views,month_before_views,year_before_views,day_after_views,week_after_views,month_after_views,is_event
0,0,Prime_Minister_of_Haiti,20220101,313,234,1.337607,36,75,345,1681,48322,95,1069,2726,1
1,1,2021%E2%80%932022_Boulder_County_fires,20220101,5643,-1,-5643.000000,591,591,591,591,591,3723,9997,20731,1


# Add category

In [ ]:
# add categories
category = pd.read_csv(f"drive/My Drive/Diploma/second_try/Code/add_category/category_0.csv")
for i in range(1, 11):
  temp = pd.read_csv(f"drive/My Drive/Diploma/second_try/Code/add_category/category_{i}.csv")
  category = pd.concat([category, temp])
print(category.shape)
category.head(3)

(108439, 2)


,Unnamed: 0,0
0,0,"('Prime_Minister_of_Haiti', 0.8670457601547241..."
1,1,"('2021%E2%80%932022_Boulder_County_fires', 0.9..."
2,2,"('Boulder_County,_Colorado', 0.996416389942169..."


In [ ]:
def convert_string_to_set(s):
    s = s.strip('()').strip('[]')  # Remove parentheses from start and end of string
    values = s.split(', ')
    return [i.strip("'") for i in values]

In [ ]:
category["0"] = category["0"].apply(lambda x: convert_string_to_set(x))
category["page_title"], category["category_score"], category["category"]=  zip(*category["0"])

In [ ]:
category = category[["page_title", "category", "category_score"]]
category = category.drop_duplicates(subset='page_title', keep="last")
category.head(3)

,page_title,category,category_score
0,Prime_Minister_of_Haiti,Geography.Regions.Americas.Central_America,0.8670457601547241
1,2021%E2%80%932022_Boulder_County_fires,Geography.Regions.Americas.North_America,0.9724247455596924
2,"Boulder_County,_Colorado",Geography.Regions.Americas.North_America,0.9964163899421692


In [ ]:
df = pd.merge(df, category, how="left", on=["page_title"])
print(df.shape)
df.head(3)

(108429, 17)


,Unnamed: 0,page_title,date,around,before,boost,same_day_views,prev_day_views,week_before_views,month_before_views,year_before_views,day_after_views,week_after_views,month_after_views,is_event,category,category_score
0,0,Prime_Minister_of_Haiti,20220101,313,234,1.337607,36,75,345,1681,48322,95,1069,2726,1,Geography.Regions.Americas.Central_America,0.8670457601547241
1,1,2021%E2%80%932022_Boulder_County_fires,20220101,5643,-1,-5643.000000,591,591,591,591,591,3723,9997,20731,1,Geography.Regions.Americas.North_America,0.9724247455596924
2,2,"Boulder_County,_Colorado",20220101,13056,1632,8.000000,2324,9566,11291,14326,70218,3932,6465,10314,1,Geography.Regions.Americas.North_America,0.9964163899421692


# Revisions events

In [ ]:
# add revisions
def modify_date(x):
  return x[:4]+x[5:7]+x[8:]

rev_ev = pd.read_csv("drive/My Drive/Diploma/second_try/Code/revisions_features/revisions_features_df.csv")
rev_ev['date'] = rev_ev['date'].apply(lambda x: modify_date(x))
rev_ev.date = rev_ev.date.astype(int)
print(rev_ev.shape)
rev_ev = rev_ev.rename({'title': 'page_title', 
                        'same_day_views': 'same_day_revs', 'prev_day_views': 'prev_day_revs', 'week_before_views': 'week_before_revs',
                        'month_before_views': 'month_before_revs', 'year_before_views': 'year_before_revs', 'week_after_views': 'week_after_revs'}, axis='columns')
rev_ev.head(2)

(9800, 9)


,Unnamed: 0,page_title,date,same_day_revs,prev_day_revs,week_before_revs,month_before_revs,year_before_revs,week_after_revs
0,0,Prime_Minister_of_Haiti,20220101,0,0,0,0,25,0
1,1,2021%E2%80%932022_Boulder_County_fires,20220101,0,0,0,0,0,0


In [ ]:
rev_ev2 = pd.read_csv("drive/My Drive/Diploma/second_try/Code/revisions_features/revisions_features_2_events.csv")
rev_ev2["0"] = rev_ev2["0"].apply(lambda x: convert_string_to_set(x))
rev_ev2["page_title"], rev_ev2["date"], rev_ev2["day_after_revs"], rev_ev2["day_after_2_revs"]=  zip(*rev_ev2["0"])
rev_ev2 = rev_ev2[["page_title", "date", "day_after_revs", "day_after_2_revs"]]
rev_ev2['date'] = rev_ev2['date'].apply(lambda x: modify_date(x))
rev_ev2.date = rev_ev2.date.astype(int)
rev_ev2 = rev_ev2.drop_duplicates(subset=['page_title', 'date'], keep="last")
print(rev_ev2.shape)
rev_ev2.head(2)

(9674, 4)


,page_title,date,day_after_revs,day_after_2_revs
0,Prime_Minister_of_Haiti,20220101,0,0
1,2021%E2%80%932022_Boulder_County_fires,20220101,0,0


In [ ]:
rev_ev = pd.merge(rev_ev, rev_ev2, how="left", on=["page_title", "date"])
rev_ev = rev_ev.drop('Unnamed: 0', axis=1)
print(rev_ev.shape)
rev_ev.head(3)

(9800, 10)


,page_title,date,same_day_revs,prev_day_revs,week_before_revs,month_before_revs,year_before_revs,week_after_revs,day_after_revs,day_after_2_revs
0,Prime_Minister_of_Haiti,20220101,0,0,0,0,25,0,0,0
1,2021%E2%80%932022_Boulder_County_fires,20220101,0,0,0,0,0,0,0,0
2,"Boulder_County,_Colorado",20220101,2,0,0,1,18,6,-1,1


# Revisions pages

In [ ]:
# add revisions
rev_pag = pd.read_csv("drive/My Drive/Diploma/second_try/Code/revisions_features/revisions_features_day_with_title.csv")
rev_pag = rev_pag.drop('rev_page', axis=1)
print(rev_pag.shape)
rev_pag.head(2)

(98565, 10)


,page_title,date,same_day_revs,prev_day_revs,week_before_revs,month_before_revs,year_before_revs,week_after_revs,day_after_revs,day_after_2_revs
0,Autism,20220601,0.0,0.0,0.0,23.0,172.0,4.0,3.0,0.0
1,Ayn_Rand,20220601,2.0,0.0,0.0,2.0,355.0,4.0,2.0,0.0


In [ ]:
revisions = rev_pag.append(rev_ev)
revisions = revisions.drop_duplicates(subset=['page_title', 'date'], keep="last")
print(revisions.shape)
revisions.head(3)

(104777, 10)


<ipython-input-13-50520b2c6bb0>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  revisions = rev_pag.append(rev_ev)


,page_title,date,same_day_revs,prev_day_revs,week_before_revs,month_before_revs,year_before_revs,week_after_revs,day_after_revs,day_after_2_revs
0,Autism,20220601,0.0,0.0,0.0,23.0,172.0,4.0,3.0,0.0
1,Ayn_Rand,20220601,2.0,0.0,0.0,2.0,355.0,4.0,2.0,0.0
2,Andre_Agassi,20220601,0.0,0.0,0.0,1.0,104.0,3.0,1.0,0.0


# Merge all

In [ ]:
df.shape

(108429, 17)

In [ ]:
df = pd.merge(df, revisions, how="left", on=["page_title", "date"])
df = df.drop('Unnamed: 0', axis=1)
print(df.shape)
df.head(2)

(108429, 24)


,page_title,date,around,before,boost,same_day_views,prev_day_views,week_before_views,month_before_views,year_before_views,...,category,category_score,same_day_revs,prev_day_revs,week_before_revs,month_before_revs,year_before_revs,week_after_revs,day_after_revs,day_after_2_revs
0,Prime_Minister_of_Haiti,20220101,313,234,1.337607,36,75,345,1681,48322,...,Geography.Regions.Americas.Central_America,0.8670457601547241,0.0,0.0,0.0,0.0,25.0,0.0,0,0
1,2021%E2%80%932022_Boulder_County_fires,20220101,5643,-1,-5643.000000,591,591,591,591,591,...,Geography.Regions.Americas.North_America,0.9724247455596924,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [ ]:
from google.colab import files
df.to_csv('views_revisions_category_events_1_year_edits_1_day.csv')
files.download('views_revisions_category_events_1_year_edits_1_day.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>